In [163]:
import requests
import json
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime as dt
import psycopg2

In [58]:
chicago_crimes = pd.read_csv("./Resources/Chicago_Crimes_2001_to_Present.csv", sep=',')
chicago_crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10224738,HY411648,09/05/2015 01:30:00 PM,043XX S WOOD ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,12.0,61.0,08B,1165074.0,1875917.0,2015,02/10/2018 03:50:01 PM,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,10224739,HY411615,09/04/2015 11:30:00 AM,008XX N CENTRAL AVE,0870,THEFT,POCKET-PICKING,CTA BUS,False,False,...,29.0,25.0,06,1138875.0,1904869.0,2015,02/10/2018 03:50:01 PM,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,11646166,JC213529,09/01/2018 12:01:00 AM,082XX S INGLESIDE AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,8.0,44.0,06,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,10224740,HY411595,09/05/2015 12:45:00 PM,035XX W BARRY AVE,2023,NARCOTICS,POSS: HEROIN(BRN/TAN),SIDEWALK,True,False,...,35.0,21.0,18,1152037.0,1920384.0,2015,02/10/2018 03:50:01 PM,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,10224741,HY411610,09/05/2015 01:00:00 PM,0000X N LARAMIE AVE,0560,ASSAULT,SIMPLE,APARTMENT,False,True,...,28.0,25.0,08A,1141706.0,1900086.0,2015,02/10/2018 03:50:01 PM,41.881903,-87.755121,"(41.881903443, -87.755121152)"


In [176]:
chicago_UE = pd.read_csv("./Resources/Chicago_UE_annual.csv", sep=',')
chicago_UE = chicago_UE.drop(columns=['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'])
chicago_UE

,Year,Labor Force,Labor Force Participation Rate,Employed,Employment Participation Rate,Unemployed,Unemployment Rate,Illinois Rate
0,2020.0,"3,617,500",63.0,"3,244,300",56.5,"373,300",10.3,9.5
1,2019.0,"3,725,500",64.7,"3,582,900",62.2,"142,600",3.8,4.0
2,2018.0,"3,733,500",64.7,"3,582,400",62.1,"151,100",4.0,4.4
3,2017.0,"3,768,300",65.2,"3,581,800",62.0,"186,500",4.9,5.0
4,2016.0,"3,806,000",65.9,"3,580,000",61.9,"226,000",5.9,5.9
5,2015.0,"3,774,200",65.3,"3,546,100",61.4,"228,100",6.0,6.0
6,2014.0,"3,782,800",65.6,"3,506,500",60.8,"276,300",7.3,7.2
7,2013.0,"3,806,700",66.2,"3,450,500",60.0,"356,300",9.4,9.1
8,2012.0,"3,786,400",66.3,"3,432,500",60.1,"353,900",9.3,9.0
9,2011.0,"3,727,900",65.7,"3,345,600",58.9,"382,300",10.3,9.8


In [113]:
chicago_UE.dropna()

,Year,Labor Force,Labor Force Participation Rate,Employed,Employment Participation Rate,Unemployed,Unemployment Rate,Illinois Rate
0,2020.0,"3,617,500",63.0,"3,244,300",56.5,"373,300",10.3,9.5
1,2019.0,"3,725,500",64.7,"3,582,900",62.2,"142,600",3.8,4.0
2,2018.0,"3,733,500",64.7,"3,582,400",62.1,"151,100",4.0,4.4
3,2017.0,"3,768,300",65.2,"3,581,800",62.0,"186,500",4.9,5.0
4,2016.0,"3,806,000",65.9,"3,580,000",61.9,"226,000",5.9,5.9
5,2015.0,"3,774,200",65.3,"3,546,100",61.4,"228,100",6.0,6.0
6,2014.0,"3,782,800",65.6,"3,506,500",60.8,"276,300",7.3,7.2
7,2013.0,"3,806,700",66.2,"3,450,500",60.0,"356,300",9.4,9.1
8,2012.0,"3,786,400",66.3,"3,432,500",60.1,"353,900",9.3,9.0
9,2011.0,"3,727,900",65.7,"3,345,600",58.9,"382,300",10.3,9.8


In [59]:
chicago_crimes = chicago_crimes.drop(columns=['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Location Description', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code', 'Updated On'])
chicago_crimes

,Primary Type,Description,Arrest,Domestic,X Coordinate,Y Coordinate,Year,Latitude,Longitude,Location
0,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,1165074.0,1875917.0,2015,41.815117,-87.670000,"(41.815117282, -87.669999562)"
1,THEFT,POCKET-PICKING,False,False,1138875.0,1904869.0,2015,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2,THEFT,OVER $500,False,True,NaN,NaN,2018,NaN,NaN,NaN
3,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,1152037.0,1920384.0,2015,41.937406,-87.716650,"(41.937405765, -87.716649687)"
4,ASSAULT,SIMPLE,False,True,1141706.0,1900086.0,2015,41.881903,-87.755121,"(41.881903443, -87.755121152)"
...,...,...,...,...,...,...,...,...,...,...
7360366,THEFT,OVER $500,False,False,1178944.0,1831384.0,2021,41.692609,-87.620476,"(41.692608621, -87.620476094)"
7360367,BATTERY,AGGRAVATED - OTHER DANGEROUS WEAPON,False,False,1176237.0,1868158.0,2021,41.793582,-87.629285,"(41.793582093, -87.629285305)"
7360368,BURGLARY,HOME INVASION,False,True,1147852.0,1884806.0,2021,41.839857,-87.732945,"(41.839857381, -87.732945262)"
7360369,THEFT,$500 AND UNDER,False,False,1149455.0,1922493.0,2021,41.943244,-87.726084,"(41.943243611, -87.726084133)"


In [60]:
chicago_crimes = chicago_crimes.set_index(['Year'])

,Primary Type,Description,Arrest,Domestic,X Coordinate,Y Coordinate,Latitude,Longitude,Location
Year,,,,,,,,,
2015,BATTERY,DOMESTIC BATTERY SIMPLE,False,True,1165074.0,1875917.0,41.815117,-87.670000,"(41.815117282, -87.669999562)"
2015,THEFT,POCKET-PICKING,False,False,1138875.0,1904869.0,41.895080,-87.765400,"(41.895080471, -87.765400451)"
2018,THEFT,OVER $500,False,True,NaN,NaN,NaN,NaN,NaN
2015,NARCOTICS,POSS: HEROIN(BRN/TAN),True,False,1152037.0,1920384.0,41.937406,-87.716650,"(41.937405765, -87.716649687)"
2015,ASSAULT,SIMPLE,False,True,1141706.0,1900086.0,41.881903,-87.755121,"(41.881903443, -87.755121152)"
...,...,...,...,...,...,...,...,...,...
2021,THEFT,OVER $500,False,False,1178944.0,1831384.0,41.692609,-87.620476,"(41.692608621, -87.620476094)"
2021,BATTERY,AGGRAVATED - OTHER DANGEROUS WEAPON,False,False,1176237.0,1868158.0,41.793582,-87.629285,"(41.793582093, -87.629285305)"
2021,BURGLARY,HOME INVASION,False,True,1147852.0,1884806.0,41.839857,-87.732945,"(41.839857381, -87.732945262)"


In [171]:
chicago_crimes = chicago_crimes.dropna()

In [170]:
primary_type = chicago_crimes['Primary Type'].unique()

In [76]:
chicago_crimes.sort_index(inplace=True)

,Primary Type,Description,Arrest,Domestic,X Coordinate,Y Coordinate,Latitude,Longitude,Location
Year,,,,,,,,,
2001,THEFT,$500 AND UNDER,False,False,1162375.0,1922517.0,41.943048,-87.678596,"(41.943048487, -87.678595897)"
2001,NARCOTICS,POSS: CANNABIS 30GMS OR LESS,True,False,1161772.0,1918732.0,41.932675,-87.680918,"(41.932674838, -87.68091809)"
2001,CRIMINAL DAMAGE,TO PROPERTY,False,False,1142330.0,1924427.0,41.948686,-87.752224,"(41.948686208, -87.752224196)"
2001,THEFT,OVER $500,True,False,1156138.0,1904522.0,41.893797,-87.702007,"(41.893797199, -87.702006888)"
2001,CRIMINAL TRESPASS,TO STATE SUP LAND,True,False,1176659.0,1888218.0,41.848619,-87.627133,"(41.848618952, -87.62713329)"
...,...,...,...,...,...,...,...,...,...
2021,THEFT,OVER $500,False,True,1158137.0,1910079.0,41.909005,-87.694513,"(41.909005481, -87.694513234)"
2021,THEFT,$500 AND UNDER,False,False,1152649.0,1927314.0,41.956410,-87.714217,"(41.956410091, -87.714216503)"
2021,ASSAULT,AGGRAVATED - HANDGUN,False,True,1193998.0,1854389.0,41.755381,-87.564610,"(41.7553814, -87.564609509)"


In [169]:
crime_count = chicago_crimes.groupby('Year')['Primary Type'].count()

In [126]:
crime_count_dict = {'Number of Crimes Committed': crime_count}
crime_count_df = pd.DataFrame(crime_count_dict)
#crime_count_df.sort_index(ascending=False)
crime_count_df.reset_index()
crime_count_df

,Number of Crimes Committed
Year,
2001,482877
2002,471528
2003,472038
2004,467202
2005,449920
2006,445547
2007,435693
2008,420033
2009,386048


In [175]:
#crime_count_df = crime_count_df.reset_index()
#crime_count_df = crime_count_df[crime_count_df.Year < 2021]
#crime_count_df = crime_count_df.drop(columns=['level_0', 'index'])
#crime_count_df =  crime_count_df.set_index('Year')
crime_count_df = crime_count_df.sort_index(ascending=False)
crime_count_df

,Number of Crimes Committed
Year,
2020,208461
2019,259136
2018,263450
2017,264939
2016,267244
2015,257906
2014,273731
2013,306365
2012,335451


In [118]:
chicago_UE['Year'] = chicago_UE['Year'].astype(int)

In [167]:
#chicago_UE = chicago_UE[chicago_UE.Year >= 2001]
#chicago_UE = chicago_UE.set_index('Year')
chicago_UE_rates = chicago_UE['Unemployment Rate']

In [168]:
UE_dict = {'Unemployement Rate': chicago_UE_rates}
UE_df = pd.DataFrame(UE_dict)

In [141]:
combined_df = pd.merge(crime_count_df, UE_df, left_index=True, right_index=True)
combined_df

,Number of Crimes Committed,Unemployement Rate
Year,,
2020,208461,10.3
2019,259136,3.8
2018,263450,4.0
2017,264939,4.9
2016,267244,5.9
2015,257906,6.0
2014,273731,7.3
2013,306365,9.4
2012,335451,9.3


In [172]:
connection = "postgres:pbarrett@localhost:5432/ETL-project"
engine = create_engine(f'postgresql://{connection}')

In [ ]:
crime_count_df.to_sql(name='Crime', con=engine, if_exists='append', index=False)

In [ ]:
UE_df.to_sql(name='Unemployment', con=engine, if_exists='append', index=False)

In [174]:
combined_df.to_csv('chicago_crime_UE')